In [16]:
import numpy as np 
import pandas as pd 
import os
import tensorflow as tf

from glob import glob
from tqdm import tqdm

import matplotlib.pyplot as plt
from PIL import Image

In [17]:
X = []
Y = []

for i in tqdm(glob('/kaggle/input/mrl-dataset/train/Open_Eyes/*')):
    temp = np.array(Image.open(i).resize((64,64)))
    X.append(temp)
    Y.append(1)
    
for i in tqdm(glob('/kaggle/input/mrl-dataset/train/Closed_Eyes/*')):
    temp = np.array(Image.open(i).resize((64,64)))
    X.append(temp)
    Y.append(0) 

100%|██████████| 2000/2000 [00:03<00:00, 546.36it/s]


In [18]:
X = np.array(X)
X = X/255.0
Y = np.array(Y)

X.shape

(4000, 64, 64)

In [19]:
X = np.expand_dims(X,-1)

X.shape

(4000, 64, 64, 1)

In [20]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Conv2D, BatchNormalization, MaxPooling2D,Dropout, Flatten

model = tf.keras.models.Sequential([
      Input(shape=(64, 64, 1)),

      Conv2D(filters = 32, kernel_size = 5, strides = 1, activation = 'relu'),
      Conv2D(filters = 32, kernel_size = 5, strides = 1, activation = 'relu', use_bias=False),
      BatchNormalization(),
      MaxPooling2D(strides = 2),
      Dropout(0.3),

      Conv2D(filters = 64, kernel_size = 3, strides = 1, activation = 'relu'),
      Conv2D(filters = 64, kernel_size = 3, strides = 1, activation = 'relu', use_bias=False),
      BatchNormalization(),
      MaxPooling2D(strides = 2),
      Dropout(0.3),

      Flatten(),
      Dense(units  = 256, activation = 'relu', use_bias=False),
      BatchNormalization(),

      Dense(units = 128, use_bias=False, activation = 'relu'),

      Dense(units = 84, use_bias=False, activation = 'relu'),
      BatchNormalization(),
      Dropout(0.3),

      Dense(units = 1, activation = 'sigmoid')
  ])

In [22]:
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])

In [23]:
callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='Sleep_Detection_Model_v1.0.keras',
    save_weights_only=False,
    monitor='val_loss',
    mode='min',
    save_best_only=True,
    verbose =1)
model.fit(x_train, y_train, validation_split=0.2, epochs=30, batch_size=32, callbacks=callback)

Epoch 1/30
64/67 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8782 - loss: 0.2734
Epoch 1: val_loss improved from inf to 0.55329, saving model to Sleep_Detection_Model_v1.0.keras
67/67 ━━━━━━━━━━━━━━━━━━━━ 10s 26ms/step - accuracy: 0.8821 - loss: 0.2658 - val_accuracy: 0.6026 - val_loss: 0.5533
Epoch 2/30
65/67 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9770 - loss: 0.0665
Epoch 2: val_loss improved from 0.55329 to 0.36407, saving model to Sleep_Detection_Model_v1.0.keras
67/67 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9770 - loss: 0.0664 - val_accuracy: 0.8358 - val_loss: 0.3641
Epoch 3/30
65/67 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9825 - loss: 0.0632
Epoch 3: val_loss improved from 0.36407 to 0.16953, saving model to Sleep_Detection_Model_v1.0.keras
67/67 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9823 - loss: 0.0631 - val_accuracy: 0.9608 - val_loss: 0.1695
Epoch 4/30
65/67 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9945 - loss: 0.0212
Epoch 4: val_los

In [24]:
model.evaluate(x_test, y_test)

42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8531 - loss: 0.8743


W0000 00:00:1718649915.562327     143 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


[0.9659162759780884, 0.8378787636756897]